In [44]:
#!pip install earthengine-api
import ee

In [45]:
# Authenticate the Earth Engine API
#ee.Authenticate()
# Initialize the API
ee.Initialize()

#Enter the generated Token on the command bar

In [48]:
# Define the start and end date
start_period = ee.Date('2024-01-01')
end_period = ee.Date('2024-08-28')

In [49]:
# Load the ESA WorldCover dataset for 2020
worldcover = ee.Image('ESA/WorldCover/v100/2020')

# Select the cropland class (cropland is class 40 in WorldCover)
cropland = worldcover.eq(40)

In [50]:
# Bitmask utility function
def bitwiseExtract(value, fromBit, toBit=None):
    if toBit is None:
        toBit = fromBit
    mask_size = ee.Number(1).add(toBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(mask_size).subtract(1)
    return value.rightShift(fromBit).bitwiseAnd(mask)


In [52]:
# Apply MODIS quality mask
def modisQA_mask(image):
    sqa = image.select('SummaryQA')
    dqa = image.select('DetailedQA')
    
    viQualityFlagsS = bitwiseExtract(sqa, 0, 1)
    viQualityFlagsD = bitwiseExtract(dqa, 0, 1)
    viSnowIceFlagsD = bitwiseExtract(dqa, 14)
    
    mask = (viQualityFlagsS.eq(0)  # Good data, use with confidence
            .And(viQualityFlagsD.eq(0))  # VI produced with good quality
            .And(viSnowIceFlagsD.eq(0)))  # No snow/ice
    
    return image.updateMask(mask)

In [53]:
# Import MODIS data collections (Terra and Aqua)
MOD13 = ee.ImageCollection('MODIS/061/MOD13Q1').filterDate(start_period, end_period)
MYD13 = ee.ImageCollection('MODIS/061/MYD13Q1').filterDate(start_period, end_period)

In [54]:
# Filter the images with the quality mask
mod13q1_QC = MOD13.map(modisQA_mask)
myd13q1_QC = MYD13.map(modisQA_mask)


In [55]:
# Merge Terra and Aqua collections
mxd13q1_cleaned = mod13q1_QC.select('EVI').merge(myd13q1_QC.select('EVI'))
#mxd13q1_cleaned_sorted = mxd13q1_cleaned.sort("system:time_start")

In [58]:
# Load ESA WorldCover dataset for 2020 (Cropland class 40)
worldcover = ee.Image('ESA/WorldCover/v100/2020')
cropland = worldcover.eq(40)

#Function to apply the cropland mask to each image
def apply_cropland_mask(image):
    return image.updateMask(cropland)

# Apply the cropland mask to the MODIS images in the collection
cropland_mxd13q1 = mxd13q1_cleaned.map(apply_cropland_mask)



In [61]:
# Visualization parameters
vis_params = {
    'min': 0.0,
    'max': 8000.0,
    'palette': [
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
        '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
        '012E01', '011D01', '011301'
    ],
}

In [67]:
# Visualize the first image on the map
import folium
#Create a Folium map centered on Syria
m = folium.Map(location=[35.0, 38.0], zoom_start=5,tiles = 'cartodb positron')

# Add Earth Engine layer
def add_ee_layer(map_object, ee_object, vis_params, name):
    map_id_dict = ee.Image(ee_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr="Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
        name=name,
        overlay=True,
        control=True
    ).add_to(map_object)

# Add the first image of the masked cropland EVI collection to the map
add_ee_layer(m, cropland_mxd13q1.first(), vis_params, 'Cropland EVI')

# Add layer control and display the map
m.add_child(folium.LayerControl())
m


In [64]:
mxd13q1_cleaned_sorted = cropland_mxd13q1.sort('system:time_start')

In [68]:
'''# Function to create export tasks for each image in the collection
def batch_export_to_drive(image_collection, folder_name, region, scale=250):
    # Get the list of image indices in the collection
    image_list = image_collection.toList(image_collection.size())
    
    # Loop through each image in the collection
    for i in range(0, image_collection.size().getInfo()):
        # Get the current image
        image = ee.Image(image_list.get(i))
        
        # Extract the date of the image for naming the file
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        
        # Define the export task
        task = ee.batch.Export.image.toDrive(
            image=image.clip(region),
            description=f'modis_evi_{date}',  # Unique description for each export task
            folder=folder_name,  # Google Drive folder name
            fileNamePrefix=f'modis_evi_{date}',  # File name prefix
            scale=scale,  # Spatial resolution in meters
            region=region.bounds().getInfo()['coordinates'],  # Export region (coordinates)
            fileFormat='GeoTIFF',
            maxPixels=1e13  # Maximum number of pixels allowed in the export
        )
        
        # Start the task
        task.start()
        #print(f"Started export task for {date}")

# Call the batch export function
batch_export_to_drive(mxd13q1_cleaned_sorted, 'Syr_mxd13q1_evi', syr_geometry)

print("All tasks started. Monitor progress in the GEE Tasks tab or Python script output.")'''

All tasks started. Monitor progress in the GEE Tasks tab or Python script output.
